# Mapzen python code

You will need to
- install Enthought or Anaconda, or another reader
- install pandas, requests and json library

## Always execute these next two cells of code!

This is a MapZen key - you will need to use your own. Please note that you should NEVER share your key to GitHub or elsewhere.

In [61]:
key = 'mapzen-RYAEveN'
endpoint = "https://matrix.mapzen.com/many_to_many"

In [62]:
import requests
import json
import pandas
import math

## Loading a keyfile

I've specificed a keyFile which stores my MapZen key so I can update this code without having to delete my key each time! And so can you. Create a json file in a text editor:

{
    
    "key":"your-key-here"
    
}

And save it as key.json somewhere not in your git directory. Then modify the below folder and it should update your key!

In [49]:
keyFolder = "/Users/swise/Projects/FTC/"
keyFile = "key.json"
with open(keyFolder+keyFile) as data_file:    
    keyJSON = json.load(data_file)
key = keyJSON['key']

# 1. Example query:

In [52]:
path=json.dumps({"locations":[{"lat":51.5079062,"lon":-0.1397546},{"lat":51.5096966,"lon":-0.1337116},{"lat":51.5096966,"lon":-0.1437116}], "costing":"pedestrian"})
#({"locations":[{"lat":40.744014,"lon":-73.990508},{"lat":40.739735,"lon":-73.979713},{"lat":40.752522,"lon":-73.985015},{"lat":40.750117,"lon":-73.983704},{"lat":40.750552,"lon":-73.993519}], "costing":"auto"})

In [53]:
#params = {"json": json.dumps(route), "api_key": key}
params = {"json":path,"api_key":key}
req = requests.get(endpoint, params=params)

In [55]:
print(req.text, "\n")
print(req.url, "\n")
print(req.json, "\n")

{'json': '{"locations": [{"lat": 51.5079062, "lon": -0.1397546}, {"lat": 51.5096966, "lon": -0.1337116}, {"lat": 51.5096966, "lon": -0.1437116}], "costing": "pedestrian"}', 'api_key': 'mapzen-RYAEveN'}


# 2. Working with a small file (fewer than 50 locations)
## Specify the filename and put it in the "data" folder

Pay attention to the format of the file...

In [88]:
filename = "Fraser_small.csv"
#filename = "Example3_fake.csv"

In [89]:
dataIn = pandas.read_csv("data/"+filename)
dataIn.head()

,Lat,Long,"lat,long"
0,51.5261,-0.2598,"51.5261,-0.2598"
1,51.5090,-0.1239,"51.509,-0.1239"
2,51.5142,-0.1185,"51.5142,-0.1185"
3,51.5135,-0.1188,"51.5135,-0.1188"
4,51.5130,-0.1242,"51.513,-0.1242"


## Create dict for locations

In [91]:
locations = []
for i in range(len(dataIn.index)):
    locations.append({"lon":dataIn.Long.iloc[i], "lat":dataIn.Lat.iloc[i]})

test1 = locations

## Specify the travel mode
- auto (car)
- pedestrian
- bicyle

In [114]:
costing = "pedestrian"

## Create a new dict with this and other prefs and feed it into a new json object

In [93]:
dictPath = {"locations":locations, 
            "costing":costing}
#print dictPath

In [94]:
path = json.dumps(dictPath)

## Send a request to MapZen

In [95]:
params = {"json":path,"api_key":key}
req = requests.get(endpoint, params=params)
req

<Response [200]>

In [97]:
print(req.url)
print(req)

https://matrix.mapzen.com/many_to_many?json=%7B%22locations%22%3A+%5B%7B%22lon%22%3A+-0.2598%2C+%22lat%22%3A+51.5261%7D%2C+%7B%22lon%22%3A+-0.1239%2C+%22lat%22%3A+51.50899999999999%7D%2C+%7B%22lon%22%3A+-0.1185%2C+%22lat%22%3A+51.5142%7D%2C+%7B%22lon%22%3A+-0.1188%2C+%22lat%22%3A+51.5135%7D%2C+%7B%22lon%22%3A+-0.1242%2C+%22lat%22%3A+51.513000000000005%7D%2C+%7B%22lon%22%3A+-0.1239%2C+%22lat%22%3A+51.5146%7D%2C+%7B%22lon%22%3A+-0.1187772%2C+%22lat%22%3A+51.5101351%7D%2C+%7B%22lon%22%3A+-0.12410339999999999%2C+%22lat%22%3A+51.5120463%7D%2C+%7B%22lon%22%3A+-0.12475080000000001%2C+%22lat%22%3A+51.509319899999994%7D%2C+%7B%22lon%22%3A+-0.117148%2C+%22lat%22%3A+51.511059%7D%2C+%7B%22lon%22%3A+-0.1213275%2C+%22lat%22%3A+51.5105677%7D%2C+%7B%22lon%22%3A+-0.117068%2C+%22lat%22%3A+51.514619%7D%2C+%7B%22lon%22%3A+-0.12297580000000001%2C+%22lat%22%3A+51.510507000000004%7D%2C+%7B%22lon%22%3A+-0.1253464%2C+%22lat%22%3A+51.510065399999995%7D%2C+%7B%22lon%22%3A+-0.1264344%2C+%22lat%22%3A+51.5092691%7D

In [98]:
jsonback = req.json()
locationsOut = jsonback.get("locations")
rawMatrix = jsonback.get("many_to_many")
rawMatrix

[[{'distance': 0.0, 'from_index': 0, 'time': 0, 'to_index': 0},
  {'distance': 18.74, 'from_index': 0, 'time': 9232, 'to_index': 1},
  {'distance': 11.456, 'from_index': 0, 'time': 8119, 'to_index': 2},
  {'distance': 11.447, 'from_index': 0, 'time': 8116, 'to_index': 3},
  {'distance': 11.116, 'from_index': 0, 'time': 7870, 'to_index': 4},
  {'distance': 11.059, 'from_index': 0, 'time': 7828, 'to_index': 5},
  {'distance': 18.779, 'from_index': 0, 'time': 9258, 'to_index': 6},
  {'distance': 11.214, 'from_index': 0, 'time': 7946, 'to_index': 7},
  {'distance': 18.843, 'from_index': 0, 'time': 9313, 'to_index': 8},
  {'distance': 18.943, 'from_index': 0, 'time': 9376, 'to_index': 9},
  {'distance': 18.821, 'from_index': 0, 'time': 9295, 'to_index': 10},
  {'distance': 11.581, 'from_index': 0, 'time': 8205, 'to_index': 11},
  {'distance': 18.979, 'from_index': 0, 'time': 9399, 'to_index': 12},
  {'distance': 11.138, 'from_index': 0, 'time': 7887, 'to_index': 13},
  {'distance': 11.047, 

## Specify Matrix

Create a pandas dataframe, which is easy to save to csv. Here we use "distance", but we could also use "time".

Here the **origin** is the **column**, and the **destination** is the **row**. The matrix is not necessarily symmetric.

In [99]:
cols = list(range(1, len(dataIn.index)))
mo = pandas.DataFrame(columns=cols, index = cols)

In [100]:
for row in rawMatrix:
    for edge in row:
        D = edge['to_index'] - 1
        O = edge['from_index'] - 1
        distance = edge['distance']
        mo.iloc[D,O]= distance
        print(D, O, distance)
mo.head()

-1 -1 0.0
0 -1 18.74
1 -1 11.456
2 -1 11.447
3 -1 11.116
4 -1 11.059
5 -1 18.779
6 -1 11.214
7 -1 18.843
8 -1 18.943
9 -1 18.821
10 -1 11.581
11 -1 18.979
12 -1 11.138
13 -1 11.047
14 -1 11.064
-1 0 18.734
0 0 0.0
1 0 0.809
2 0 0.744
3 0 0.612
4 0 0.84
5 0 0.457
6 0 0.433
7 0 0.096
8 0 0.626
9 0 0.275
10 0 0.907
11 0 0.232
12 0 0.23
13 0 0.214
14 0 0.293
-1 1 11.456
0 1 0.809
1 1 0.0
2 1 0.248
3 1 0.562
4 1 0.502
5 1 0.544
6 1 0.598
7 1 0.824
8 1 0.668
9 1 0.533
10 1 0.329
11 1 0.691
12 1 0.846
13 1 0.942
14 1 1.022
-1 2 11.447
0 2 0.744
1 2 0.248
2 2 0.0
3 2 0.553
4 2 0.493
5 2 0.479
6 2 0.535
7 2 0.759
8 2 0.603
9 2 0.468
10 2 0.363
11 2 0.547
12 2 0.728
13 2 0.781
14 2 0.86
-1 3 11.116
0 3 0.612
1 3 0.562
2 3 0.553
3 3 0.0
4 3 0.223
5 3 0.664
6 3 0.217
7 3 0.59
8 3 0.817
9 3 0.413
10 3 0.797
11 3 0.437
12 3 0.461
13 3 0.531
14 3 0.624
-1 4 11.059
0 4 0.84
1 4 0.502
2 4 0.493
3 4 0.223
4 4 0.0
5 4 0.707
6 4 0.406
7 4 0.817
8 4 0.826
9 4 0.602
10 4 0.737
11 4 0.626
12 4 0.689
13 4 0.6

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,0,0.809,0.744,0.612,0.84,0.457,0.433,0.096,0.626,0.275,0.907,0.232,0.23,0.214,0.293
2,0.809,0,0.248,0.562,0.502,0.544,0.598,0.824,0.661,0.533,0.329,0.691,0.846,0.942,1.022
3,0.744,0.248,0,0.553,0.493,0.479,0.535,0.759,0.596,0.468,0.363,0.547,0.728,0.781,0.86
4,0.612,0.562,0.553,0,0.223,0.664,0.217,0.59,0.817,0.413,0.797,0.437,0.461,0.531,0.624
5,0.84,0.502,0.493,0.223,0,0.707,0.406,0.817,0.826,0.602,0.737,0.626,0.689,0.699,0.792


In [101]:
mo.to_csv("dataOut/"+filename)

# 3. Larger files (more than 50 locations)

In [31]:
#filename = "Example4_fake"
filename = "Chenchen_26"

In [56]:
dataIn = pandas.read_csv("data/"+filename+".csv")
print(dataIn)

          lat       lon
0   51.526110 -0.259841
1   51.513281 -0.144123
2   51.511824 -0.143748
3   51.511057 -0.139409
4   51.508820 -0.136439
5   51.509486 -0.137171
6   51.508684 -0.137453
7   51.508663 -0.141459
8   51.507062 -0.141558
9   51.507751 -0.141671
10  51.508684 -0.140394
11  51.507906 -0.139755
12  51.509509 -0.141872
13  51.507975 -0.141100
14  51.508163 -0.139755
15  51.510968 -0.142297
16  51.511389 -0.140522
17  51.511589 -0.140098
18  51.511335 -0.138999
19  51.512943 -0.141995
20  51.513149 -0.141840
21  51.513108 -0.141084
22  51.513708 -0.141488
23  51.514370 -0.146954
24  51.513801 -0.147187
25  51.513958 -0.147444
26  51.514269 -0.145603
27  51.512297 -0.142068
28  51.509479 -0.136080
29  51.509693 -0.137430
30  51.509642 -0.135649
31  51.508675 -0.140008
32  51.509697 -0.133712
33  51.508200 -0.138881
34  51.512300 -0.140876
35  51.515741 -0.143728
36  51.514229 -0.141995
37  51.512209 -0.144260
38  51.512312 -0.143961
39  51.512637 -0.143162
40  51.510850 -0

## Split this into multiple location lists (a list of lists)

Make sure your column names match up here

In [81]:
nPerBin = 20
breakup = 1 + int(math.floor(len(dataIn)/float(nPerBin)))
print("breakup")
print(breakup)
print(type(breakup))

locations = []
for i in range(0, breakup):
    locations.append([])
    
#latcol = "Delivery Lat"
#loncol = 'Deliver Long'

latcol = "lat"
loncol = "lon"

n=0
for i in range(len(dataIn.index)):
    n = (int)(math.floor(i/nPerBin))
    locations[n].append({"lon":dataIn[loncol].iloc[i], "lat":dataIn[latcol].iloc[i]})

breakup
3
<class 'int'>
[{'lon': -0.14108380000000001, 'lat': 51.513107599999998}, {'lon': -0.1414879, 'lat': 51.513707700000005}, {'lon': -0.1469539, 'lat': 51.514370299999996}, {'lon': -0.14718720000000002, 'lat': 51.513801100000002}, {'lon': -0.1474442, 'lat': 51.5139578}, {'lon': -0.14560329999999999, 'lat': 51.514269499999997}, {'lon': -0.142068, 'lat': 51.512297400000001}, {'lon': -0.1360797, 'lat': 51.5094785}, {'lon': -0.13742970000000002, 'lat': 51.509692700000002}, {'lon': -0.13564899999999999, 'lat': 51.509641999999999}, {'lon': -0.14000760000000001, 'lat': 51.508674600000006}, {'lon': -0.13371159999999999, 'lat': 51.509696599999998}, {'lon': -0.1388808, 'lat': 51.5081998}, {'lon': -0.1408761, 'lat': 51.512299799999994}, {'lon': -0.14372799999999999, 'lat': 51.515741200000001}, {'lon': -0.14199520000000002, 'lat': 51.514228600000003}, {'lon': -0.14425959999999999, 'lat': 51.512208600000001}, {'lon': -0.14396110000000001, 'lat': 51.512312000000001}, {'lon': -0.1431625, 'lat':

## Save into one .csv

Called "FilenameIn_out.csv"

In [84]:
costing = "pedestrian"
endpoint = "https://matrix.mapzen.com/sources_to_targets"

bigDF = pandas.DataFrame()

for n in range (0, breakup):
    colDF=pandas.DataFrame()
    
    for m in range(0, n-1):
        cols = list(range(1+n*nPerBin, 1+n*nPerBin+len(locations[n])))
        rows = list(range(1+m*nPerBin, 1+m*nPerBin+len(locations[m])))
        mo = pandas.DataFrame(columns=cols, index=rows)
        if m==0:
            colDF = mo
        else:
            colDF = colDF.append(mo)

    for m in range (n, breakup):
        dictPath = {"sources":locations[n],
                    "targets":locations[m],
                    "costing":costing,
                    "id": filename}
        path = json.dumps(dictPath)
        params = {"json":path,"api_key":key}
        req = requests.get(endpoint, params=params)
        jsonback = req.json()
        #locationsOut = jsonback.get("locations")
        rawMatrix = jsonback.get("sources_to_targets")
        
        cols = list(range(1+n*nPerBin, 1+n*nPerBin+len(locations[n])))
        rows = list(range(1+m*nPerBin, 1+m*nPerBin+len(locations[m])))
        mo = pandas.DataFrame(columns=cols, index=rows)
        
        #print(rawMatrix)
        if(rawMatrix is not None):
            for row in rawMatrix:
                for edge in row:
                    D = edge['to_index']
                    O = edge['from_index']
                    distance = edge['distance']
                    mo.iloc[D,O]= distance
#                    print(locations[n][O], locations[m][D], distance)
        else:
            print("badrequest apparently")
            
        #build up matrix for this n to all m
        if m==0:
            colDF = mo
        else:
            colDF = colDF.append(mo)
            
    if n==0:
        bigDF = colDF
    else:
        bigDF = bigDF.merge(colDF, left_index=True, right_index=True, how='outer') 
            
        #mo.to_csv("dataOut/"+filename+"_"+str(n)+"_"+str(0)+".csv")
bigDF.to_csv("dataOut/"+filename+"_bigBit_out.csv")

In [15]:
bigDF.tail()     

""


## JUNK BELOW - ignore it

In [231]:
costing = "auto"
endpoint = "https://matrix.mapzen.com/sources_to_targets"

dictPath = {"sources":locations[1],
            "targets":locations[1],
            "costing":costing}
print dictPath
path = json.dumps(dictPath)
params = {"json":path,"api_key":key,"id": filename}
req = requests.get(endpoint, params=params)
#print req.url

{'sources': [{'lat': 51.523625399999993, 'lon': -0.13362449999999998}, {'lat': 51.524465399999997, 'lon': -0.13269420000000001}, {'lat': 51.524188000000002, 'lon': -0.13243099999999999}, {'lat': 51.518244000000003, 'lon': -0.12096510000000001}, {'lat': 51.518924669999997, 'lon': -0.13180378000000001}, {'lat': 51.519795000000002, 'lon': -0.1313291}, {'lat': 51.518234020000001, 'lon': -0.12843228000000001}, {'lat': 51.523468700000002, 'lon': -0.12731389999999998}], 'targets': [{'lat': 51.523625399999993, 'lon': -0.13362449999999998}, {'lat': 51.524465399999997, 'lon': -0.13269420000000001}, {'lat': 51.524188000000002, 'lon': -0.13243099999999999}, {'lat': 51.518244000000003, 'lon': -0.12096510000000001}, {'lat': 51.518924669999997, 'lon': -0.13180378000000001}, {'lat': 51.519795000000002, 'lon': -0.1313291}, {'lat': 51.518234020000001, 'lon': -0.12843228000000001}, {'lat': 51.523468700000002, 'lon': -0.12731389999999998}], 'costing': 'auto'}


In [232]:
costing = "auto"
endpoint = "https://matrix.mapzen.com/many_to_many"

loc = locations[1]

dictPath = {"locations":loc,
            "costing":costing}
print dictPath
path = json.dumps(dictPath)
params = {"json":path,"api_key":key,"id": filename}
req = requests.get(endpoint, params=params)
#print req.url

{'locations': [{'lat': 51.523625399999993, 'lon': -0.13362449999999998}, {'lat': 51.524465399999997, 'lon': -0.13269420000000001}, {'lat': 51.524188000000002, 'lon': -0.13243099999999999}, {'lat': 51.518244000000003, 'lon': -0.12096510000000001}, {'lat': 51.518924669999997, 'lon': -0.13180378000000001}, {'lat': 51.519795000000002, 'lon': -0.1313291}, {'lat': 51.518234020000001, 'lon': -0.12843228000000001}, {'lat': 51.523468700000002, 'lon': -0.12731389999999998}], 'costing': 'auto'}


## Generate multiple matrices and save each to a csv

In [28]:
costing = "auto"
endpoint = "https://matrix.mapzen.com/sources_to_targets"

for n in range (0,breakup):
    for m in range (0,breakup):
        dictPath = {"sources":locations[n],
                    "targets":locations[m],
                    "costing":costing,
                    "id": filename}
        path = json.dumps(dictPath)
        params = {"json":path,"api_key":key}
        req = requests.get(endpoint, params=params)
        #print req.url
        jsonback = req.json()
        #locationsOut = jsonback.get("locations")
        rawMatrix = jsonback.get("sources_to_targets")
        cols = list(range(1+n*50, 51+n*50))
        rows = list(range(1+m*50, 51+m*50))
        mo = pandas.DataFrame(columns=cols, index = rows)
        
        for row in rawMatrix:
            for edge in row:
                D = edge['to_index']
                O = edge['from_index']
                distance = edge['distance']
                mo.iloc[D,O]= distance
                
        mo.to_csv("dataOut/"+filename+"_"+str(n)+"_"+str(0)+".csv")
        

In [167]:
costing = "pedestrian"
endpoint = "https://matrix.mapzen.com/sources_to_targets"

bigDF = pandas.DataFrame()

n = 0
m = 8

colDF=pandas.DataFrame()
dictPath = {"sources":locations[n],
                    "targets":locations[m],
                    "costing":costing,
                    "id": filename}
path = json.dumps(dictPath)


params = {"json":path,"api_key":key}
req = requests.get(endpoint, params=params)
#print(req.url)

jsonback = req.json()
rawMatrix = jsonback.get("sources_to_targets")
        


In [168]:
for row in rawMatrix:
    for edge in row:
        print(edge)

{'distance': 9.909, 'time': 7014, 'to_index': 0, 'from_index': 0}
{'distance': 0.44, 'time': 314, 'to_index': 0, 'from_index': 1}
{'distance': 0.381, 'time': 270, 'to_index': 0, 'from_index': 2}
{'distance': 0.322, 'time': 229, 'to_index': 0, 'from_index': 3}
{'distance': 0.425, 'time': 300, 'to_index': 0, 'from_index': 4}


In [169]:
print(list(range(1+n*nPerBin, 1+n*nPerBin+len(locations[n]))))
print(list(range(1+m*nPerBin, 1+m*nPerBin+len(locations[m]))))

[1, 2, 3, 4, 5]
[41]


In [165]:
print(locations[m])

[{'lon': -0.13564899999999999, 'lat': 51.509641999999999}, {'lon': -0.14000760000000001, 'lat': 51.508674600000006}, {'lon': -0.13371159999999999, 'lat': 51.509696599999998}, {'lon': -0.1388808, 'lat': 51.5081998}, {'lon': -0.1408761, 'lat': 51.512299799999994}, {'lon': -0.14372799999999999, 'lat': 51.515741200000001}, {'lon': -0.14199520000000002, 'lat': 51.514228600000003}, {'lon': -0.14425959999999999, 'lat': 51.512208600000001}, {'lon': -0.14396110000000001, 'lat': 51.512312000000001}, {'lon': -0.1431625, 'lat': 51.512636799999996}, {'lon': -0.1400295, 'lat': 51.510849900000004}]


In [58]:
range(0,0)

range(0, 0)

In [60]:
list(range(0,1))

[0]